<a href="https://colab.research.google.com/github/nachoacev/practice-data-science/blob/main/IntermediateML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We will accelerate our machine learning expertise by learning how to:

- Tackle data types often found in real-world datasets (**missing values, categorical variables**).
- Design **pipelines** to improve the quality of your machine learning code.
- Use advanced techniques for model validation (**cross-validation**).
- Build state-of-the-art models that are widely used to win Kaggle competitions (**XGBoost**).
- Avoid common and important data science mistakes (**leakage**).

Along the way, we'll apply our knowledge by completing a **hands-on exercise with real-world data** for each new topic. The hands-on exercises use data from the Housing Prices Competition for Kaggle Learn Users, where you'll use 79 different explanatory variables (such as the type of roof, number of bedrooms, and number of bathrooms) to predict home prices. You'll measure your progress by submitting predictions to this competition and watching your position rise on the leaderboard!

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 5)

# Import data
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dansbecker/home-data-for-ml-course")

path_train = "/content/train.csv"
path_test = "/content/test.csv"

print("Path to dataset files for training:", path_train)
print("Path to dataset files for testing:", path_test)

Path to dataset files for training: /content/train.csv
Path to dataset files for testing: /content/test.csv


We prepare the training and validation datas.

In [2]:
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv(path_train, index_col='Id')
X_test_full = pd.read_csv(path_test, index_col='Id')

# Obtain target and predictors
y = X_full.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF',
            'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test = X_test_full[features].copy()

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [26]:
X_train

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
619,11694,2007,1828,0,2,3,9
871,6600,1962,894,0,1,2,5
...,...,...,...,...,...,...,...
560,3196,2003,1557,0,2,2,7
685,16770,1998,1195,644,2,4,7


The next code cell defines five different random forest models. Run this code cell without changes.

In [27]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [28]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 24015
Model 2 MAE: 23740
Model 3 MAE: 23528
Model 4 MAE: 23996
Model 5 MAE: 23706


We see that the best model is `model_3`. We will add more trees for this one and see what we get:

In [37]:
# Define a model
my_model = RandomForestRegressor(n_estimators=200, criterion='absolute_error', random_state=0)

# Fit the model to the training data
my_model.fit(X, y)

# Generate test predictions
preds_test = my_model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})

# output.to_csv('submission.csv', index=False)
output

,Id,SalePrice
0,1461,120100.04
1,1462,156445.50
...,...,...
1457,2918,131768.75
1458,2919,230416.45


# Missing Values `SimpleImputer`

We will learn three approaches to __dealing with missing values__. Then we will compare the effectiveness of these approaches on a real-world dataset.

There are many ways data can end up with missing values. For example,
- A 2 bedroom house won't include a value for the size of a third bedroom.
- A survey respondent may choose not to share his income.

Most machine learning libraries (including scikit-learn) **give an error if you try to build a model using data with missing values**. So we'll need to choose one of the strategies below.

1) A Simple Option: **Drop Columns with Missing Values (worst option)**.

The simplest option is to drop columns with missing values.


| Bed  | Bath |         | Bath |
|------|------|         |------|
| 1.0  | 1.0  |         | 1.0  |
| 2.0  | 1.0  |         | 1.0  |
| 3.0  | 2.0  |         | 2.0  |
| NaN  | 2.0  |  ---->  | 2.0  |


Unless most values in the dropped columns are missing, **the model loses access to a lot of (potentially useful!) information** with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

2) A Better Option: **Imputation**

**Imputation fills in the missing values with some number**. For instance, we can fill in *the mean value along each column*.

| Bed  | Bath |         | Bed  | Bath |
|------|------|---------|------|------|
| 1.0  | 1.0  |         | 1.0  | 1.0  |
| 2.0  | 1.0  |         | 2.0  | 1.0  |
| 3.0  | 2.0  |         | 3.0  | 2.0  |
| NaN  | 2.0  |   ----> | 2.0  | 2.0  |

The imputed value won't be exactly right in most cases, but it usually **leads to more accurate models than you would get from dropping** the column entirely.

3) **An Extension To Imputation**

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

| Bed  | Bath |       | Bed  | Bath | Bed_was_missing |
|------|------|------|------|------|----------------|
| 1.0  | 1.0  |      | 1.0  | 1.0  | FALSE          |
| 2.0  | 1.0  |      | 2.0  | 1.0  | FALSE          |
| 3.0  | 2.0  |      | 3.0  | 2.0  | FALSE          |
| NaN  | 2.0  | ---->| 2.0  | 2.0  | TRUE           |

In this approach, __we impute the missing values__, as before. And, additionally, for each column with missing entries in the original dataset, __we add a new column that shows the location of the imputed entries__.

__In some cases, this will meaningfully improve results. In other cases, it doesn't help at all__.

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dansbecker/melbourne-housing-snapshot") + '/melb_data.csv'

print("Path to dataset files:", path)

100%|██████████| 451k/451k [00:00<00:00, 71.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/dansbecker/melbourne-housing-snapshot/versions/5/melb_data.csv


In [8]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv(path)

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

**Score from Approach 1 (Drop Columns with Missing Values)**

Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.

In [14]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from approach 1 (Drop columns with missing values):
183550.22137772635


**Score from Approach 2 (Imputation)**

Next, we use `SimpleImputer` **to replace missing values with the mean value along each column**.

This is a function with two important methods:
- `.fit_transform()`: it create and fit the imputate transformation from the data given.
- `.transform()`: it transform according with the fitted imputation.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as **regression imputation**, for instance), the **complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models**.

In [15]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


**Score from Approach 3 (An Extension to Imputation)**

Next, we impute the missing values, while also keeping track of which values were imputed.

In [16]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


We see that **Approach 2** has lower MAE than **Approach 1** and **Approach 3**, so Approach 2 performed better on this dataset.

**So, why did imputation perform better than dropping the columns?**

The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [17]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


__Conclusion__

As is common, imputing missing values (in **Approach 2** and **Approach 3**) yielded better results, relative to when we simply dropped columns with missing values (in **Approach 1**).

In [26]:
missing_val_count_by_column.sum()

9512